In [1]:
import numpy as np
import pandas as pd
import gc
import string
import re
import time as t
import pickle
from scipy.sparse import csc_matrix

from sqlalchemy import create_engine
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt


from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier, plot_importance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.model_selection import iterative_train_test_split, IterativeStratification

pd.options.display.max_colwidth=1000

In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [173]:

t1=t.clock()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [174]:
from tensorflow.keras.models import  Model
from tensorflow.keras import layers, Input
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [175]:
#engine = create_engine(f'mssql+pyodbc://{server}/{db}?trusted_connection=yes&driver=SQL Server Native Client 11.0') 
#df =  pd.read_sql(sql,engine)
#df.to_csv('Part_reco_phase2.csv',index=False)

In [176]:
#df = pd.read_csv('Part_reco_csdw_26Nov.csv')
current_region = "LA"
date = "25Feb"
df = pd.read_csv(f'F:\Akhilesh_new\Part_reco_regionwise\Data Preparation\Part_reco_RRlogic_{date}_{current_region}.csv')
df = df[df['Repair_Flag']==1]
df.shape

(3130, 96)

In [177]:
df_commodity = pd.read_csv(f'F:\Akhilesh_new\Part_reco_regionwise\Part_Commodity_{date}.csv')

In [178]:
df_pmaster = pd.read_excel('F:\Akhilesh_new\Part_reco_regionwise\pmaster.xlsx')

In [179]:
#df[['Product_Number','Product_Big_Series_Name','Product_Line_Code_Name','PL_Product_Category']].isna().sum()

In [180]:
df = df.drop(columns=['Product_Big_Series_Name','Product_Line_Code_Name','PL_Product_Category'])
df = df.merge(df_pmaster,how='left',on=['Product_Number'])
df = df.rename(columns={'PL_Category':'PL_Product_Category'})
df[['Product_Number','Product_Big_Series_Name','Product_Line_Code_Name','PL_Product_Category']].isna().sum()

Product_Number              0
Product_Big_Series_Name    44
Product_Line_Code_Name     44
PL_Product_Category        44
dtype: int64

In [181]:
test_period = ['202011','202012','202101']

#### Commodity mapping

In [182]:
df_model = df.merge(df_commodity,how='left',on=['Part_Number'])

#print(df_model.part_commodity.value_counts())
#print(df_model.part_commodity.isna().sum())

In [183]:
df_model.Closed_Date.min(), df_model.Closed_Date.max()

('2019-11-06 19:08:11', '2021-01-30 23:59:22')

In [184]:
pd.options.display.max_columns=100

#### Data filter

In [185]:
del df
gc.collect()

1865204

In [186]:
df_model['Closed_Date'] = pd.to_datetime(df_model['Closed_Date']).dt.strftime("%Y-%m-%d")
df_model.head()

,Case_ID,Source_x,Event_Count,Created_Date,Closed_Date,Case_Status,Region,Sub_Region_1_Name,Market_Segment,Business_Segment,Business_Subsegment,Global_Business_Unit,Product_Number,Premium_Flag,Case_OTC,Incoming_Channel,DNAP_Flag,MPS_Flag,Case_Resolution,Delivery_Alternative,Irrelevant_Name,Complaint_Flag,Case_Subject,Rerepair30days,Rerepair90days,CSR_Count,WO_Count,SO_Count,Delivery_Motion,Serial_Number,Ops_Interventions,Onsite_Service_Events,CSR_Service_Events,Product_Family,Product_Family_Description,Created_By_Workgroup,QA_Code,ADP_Flag,Market_SubSegment,Service_Flag,Case_Entitlement,Entitlement_Start_Date,Entitlement_End_Date,Executive_OTC,Executive_Category,Case_Type,Product_Description,Product_Series_Name,Entitlement_Status,More_Than_1Repair_Flag,Operating_System_Name,Recommended_Parts,Parts Canceled,DOA,Returned Good,Used & Not Consumed,Used & Consumed,No_Failure_Code,Parent_Case,Entitlement_Type,First_Part_Ordered,Last_Part_Ordered,Wrong_Parts_Recommendation_Flag,ip1,ip2,ip3,First_Closed_Date,Rerepair_days,Repair_days_le_90,FTF_plus_Maxrepair,Source_y,SubID,Part_Order_Number,PO_Open_Date,Part_Requested_Date,Part_Number,Part_Order_Detail_Description,Selected_Quantity,Failure_Code,Original_Part_Number,Offered_Part_Number,Part_Delivered,Part_Status,Subassembly,Subsystem,RR90_Same_Part_Number_Flag,RR90_Same_Commodity_Flag,RR30_Same_Part_Number_Flag,RR30_Same_Commodity_Flag,Consumed_Flag1,Consumed_Flag2,Remote_Flag,Repair_Flag,Product_Big_Series_Name,Product_Line_Code_Name,PL_Product_Category,part_commodity,part_description
0,5039628069,CDAX,1,2019-11-05 16:56:12,2019-11-08,Closed,AMS,LA,Commercial,Personal Systems,Business PC Solutions,Commercial Compute,1ZR97LT,0,05K-Extended Warranty,Phone,0,0,Onsite Solution,Subcontractor Onsite,0.0,0,PE/ONS/NCD/440 G5/Problemas de teclado y stickpoint,NaN,NaN,0.0,1.0,0.0,Onsite,5CD7517GR1,1.0,NaN,1.0,FELB,Ratchet STD 1.0 -HP ProBook 440 G5,LLP_SC GSC CRA H PSG LAC,NaN,0,Notebooks,NBD,carepack (XLS9B6M742C3),2018-03-28,2021-03-27,Trade,MG/CY,HW Delivery,HP ProBook 440 G5 Notebook PC,HP ProBook 440 G5 Notebook PC,Active,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,Care-Pack,L01072-161,L01072-161,0,input_output_device_hp_only,keyboard,some_keys_not_working_(keyboard),2019-11-08 23:01:43,-4,1,1.0,CDAX,WO-008885924,PO-111944361 - 1,2019-11-05 17:07:52,2019-11-05 23:00:00,L01072-161,SPS-KBD CP SR CHILE,1.0,73 - Used & Consumed,L01072-161,L01072-161,NaN,Shipped,Keyboard,I/O DEVICE,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,HP ProBook 440 G5 Notebook PC (Ratchet),6U,COMMERCIAL NOTEBOOKS,Keyboard,SPS-KBD CP SR CHILE
1,5039703165,CDAX,1,2019-11-06 20:25:08,2019-11-20,Closed,AMS,LA,Commercial,Personal Systems,Business PC Solutions,Commercial Compute,3WQ23EC,0,05R-Normal Contract,Phone,0,0,Parts Shipped,CSR,0.0,0,BS/CSR/ONS/NCD/840 G5/Problemas de Cargador,1.0,1.0,2.0,0.0,0.0,CSR,5CG83549K0,2.0,2.0,NaN,FETR,Computron14 PRO 1.0 -EliteBook840/846G5,LLP_SC GSC CRA H PSG LAC,NaN,0,Notebooks,NBD,contract (104898957774),2018-10-01,2021-09-30,Trade,6700,HW Delivery,HP EliteBook 840 G5 Notebook PC,HP EliteBook 840 G5 Notebook PC,Active,1.0,NaN,2.0,0.0,0.0,0.0,0.0,2.0,0.0,NaN,Contract,860209-850,860209-850,0,power,wont_power_on,no_power_no_light_on_notebook,2019-11-20 17:52:52,-14,1,1.0,CDAX,CSR-101451186,PO-112022311 - 1,2019-11-11 14:27:33,NaN,860209-850,GNRC-65W ADPTR nPFC RC USB-C 3PIN,1.0,73 - Used & Consumed,NaN,860209-850,NaN,Shipped,AC Adapter,POWER,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,HP EliteBook 840/846 G5 Notebook PC (Computron14),AN,COMMERCIAL NOTEBOOKS,PwrSuply/Mains/Adptr,GNRC-65W ADPTR nPFC RC USB-C 3PIN
2,5039703165,CDAX,1,2019-11-06 20:25:08,2019-11-20,Closed,AMS,LA,Commercial,Personal Systems,Business PC Solutions,Commercial Compute,3WQ23EC,0,05R-Normal Contract,Phone,0,0,Parts Shipped,CSR,0.0,0,BS/CSR/ONS/NCD/840 G5/Problemas de Cargador,1.0,1.0,2.0,0.0,0.0,CSR,5CG83549K0,2.0,2.0,NaN,FETR,Computron14 PRO 1.0 -EliteBook840/846G5,LLP_SC GSC CRA H PSG LAC,NaN,0,Notebooks,NBD,contract (104898957774),2018-10-01,

In [187]:
#df_model['Market_SubSegment'].value_counts()

In [188]:

df_model = df_model[df_model['Closed_Date']>='2019-11-01']
df_model = df_model[df_model['Closed_Date']<'2021-02-01']

In [189]:
df_model.Closed_Date.min(), df_model.Closed_Date.max()

('2019-11-06', '2021-01-30')

In [190]:
#Case OTC filtering
#df_model = df_model[df_model['Case_OTC'].isin(
#    ['02N-Normal Warranty','05K-Extended Warranty','00E-No Entitlement','05D-ADP Coverage',
#     '05R-Normal Contract','00C-Claims Contract','01-Trade','00W-Claims Warranty'])]

In [191]:
#Case Resolution filtering
df_model = df_model[~(df_model['Case_Resolution'].isin(['Cancel per Customer','Case Voided','Duplicate Case']))]

In [192]:
df_model = df_model[df_model['Delivery_Motion'].isin(['Onsite','CSR'])]

In [193]:
df_model = df_model[df_model['Incoming_Channel'].isin(['Phone'])]

In [194]:
df_model = df_model[df_model['Market_SubSegment']=='Notebooks']

In [195]:
print(df_model.part_commodity.value_counts())

PCA I/O                 409
Keyboard                406
Hard Drive              233
Battery                 217
PwrSuply/Mains/Adptr    146
Display                 110
Fan/Motor                87
Solid State Drive        74
Cable/Adaptr/PwrCord     73
Human Input Device       48
Chassis/Enclosure        48
Memory/Firmware          40
Misc Hardware            36
PCA Misc                 34
Electronic Component     28
PCA Network               7
Scanner Assy              2
Removable Media           1
Optical Drive             1
CPU                       1
Name: part_commodity, dtype: int64


In [196]:
#Data cleaning on Part number to be done
df_model.shape, df_model.part_commodity.isna().sum()

((2006, 98), 5)

In [197]:
#df_model = df_model[df_model['Used & Consumed']<=3]
print(df_model.shape)

(2006, 98)


In [198]:
df_model = df_model[~(df_model['part_commodity'].isna())]
df_model = df_model[df_model['part_commodity']!='NULL']
df_model.shape

(2001, 98)

In [199]:
threshold=0.995
cumsum_comm = df_model.part_commodity.value_counts(normalize=True).cumsum()<threshold
comm_idx = len(cumsum_comm[cumsum_comm > threshold])
comm_name = list(cumsum_comm.iloc[comm_idx:].index)
print(comm_name)
df_model = df_model[~(df_model['part_commodity'].isin(comm_name))]
print(df_model.shape)

['PCA Network', 'Scanner Assy', 'Removable Media', 'Optical Drive', 'CPU']
(1989, 98)


#### Case subject refinement

In [200]:
'''
chars = string.ascii_lowercase
#df_model = df.copy()
df_model = df_model.dropna(subset=['Case_Subject'])
df_model['case_subject_refined'] = df_model['Case_Subject'].apply(lambda x: str(x).strip().lower())

df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: re.sub('\s+',' ',x))
df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: re.sub('\/*\s*$',' ',x))
df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: x.split(r'/')[-1])
#df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: x.split(r'-')[-1])
df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: ''.join([char if char in chars else ' ' for char in x ]))
#df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split() if len(word)>2]))
df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: ' '.join([word for word in x.split() if len(word)>=2]))
#df_model.head()
'''

"\nchars = string.ascii_lowercase\n#df_model = df.copy()\ndf_model = df_model.dropna(subset=['Case_Subject'])\ndf_model['case_subject_refined'] = df_model['Case_Subject'].apply(lambda x: str(x).strip().lower())\n\ndf_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: re.sub('\\s+',' ',x))\ndf_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: re.sub('\\/*\\s*$',' ',x))\ndf_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: x.split(r'/')[-1])\n#df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: x.split(r'-')[-1])\ndf_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: ''.join([char if char in chars else ' ' for char in x ]))\n#df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split() if len(word)>2]))\ndf_model['case_subject_refined'] = df_model['case_subject_re

In [201]:
chars = string.ascii_lowercase
df_model['case_subject_refined'] = df_model['Case_Subject'].apply(lambda x: str(x).strip().lower())

df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: re.sub('\s+',' ',x))
df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: re.sub('\/*\s*$',' ',x))
df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: ' '.join([x.split(r'/')[-2],x.split(r'/')[-1]]) if len(x.split(r'/'))>2 else x.split(r'/')[-1])
df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: ''.join([char if char in chars else ' ' for char in x ]))
df_model['case_subject_refined'] = df_model['case_subject_refined'].apply(lambda x: ' '.join([word for word in x.split() if len(word)>=2]))


In [202]:
df_model[['Case_Subject','case_subject_refined']].head()

,Case_Subject,case_subject_refined
0,PE/ONS/NCD/440 G5/Problemas de teclado y stickpoint,problemas de teclado stickpoint
1,BS/CSR/ONS/NCD/840 G5/Problemas de Cargador,problemas de cargador
2,BS/CSR/ONS/NCD/840 G5/Problemas de Cargador,problemas de cargador
3,TT/BENCH/5BD/450 G5/No power,no power
5,MX/ONS/NCD/840 G5/Problemas de teclado y stickpoint,problemas de teclado stickpoint


#### Feature Engineering

In [203]:
df_model['Entitlement_Start_Date'] = pd.to_datetime(df_model['Entitlement_Start_Date'])
df_model['Entitlement_End_Date'] = pd.to_datetime(df_model['Entitlement_End_Date'])
df_model['Created_Date'] = pd.to_datetime(df_model['Created_Date'])
df_model['Closed_Date'] = pd.to_datetime(df_model['Closed_Date'])
df_model['Entitlement_Age_from_start'] = ((df_model.Created_Date - df_model.Entitlement_Start_Date)/np.timedelta64(1,'M')).fillna(0).astype('int')
df_model['Entitlement_Age_from_end'] = ((df_model.Created_Date - df_model.Entitlement_End_Date)/np.timedelta64(1,'M')).fillna(0).astype('int')

df_model['Closed_Date'] = df_model.Closed_Date.dt.strftime('%Y%m').astype('int')

#### Feature Selection

In [204]:
#Ignored Incomming_Channel
final_features = ['Case_ID','Product_Big_Series_Name', 'Product_Line_Code_Name', 'PL_Product_Category', 'Product_Number','Used & Consumed','Failure_Code'
                  ,'Recommended_Parts','Serial_Number','Entitlement_Type','Entitlement_Age_from_start','Entitlement_Age_from_end','Entitlement_Status','case_subject_refined','Closed_Date','part_commodity',
                 'ip1','ip2','ip3']


df_final2 = df_model[final_features]

In [205]:
df_final2.isna().sum()

Case_ID                        0
Product_Big_Series_Name       19
Product_Line_Code_Name        19
PL_Product_Category           19
Product_Number                 0
Used & Consumed                0
Failure_Code                   0
Recommended_Parts              0
Serial_Number                  0
Entitlement_Type               3
Entitlement_Age_from_start     0
Entitlement_Age_from_end       0
Entitlement_Status             3
case_subject_refined           0
Closed_Date                    0
part_commodity                 0
ip1                            0
ip2                            1
ip3                            1
dtype: int64

In [206]:
df_final2['Product_Big_Series_Name'] = df_final2['Product_Big_Series_Name'].str.lower()
df_final2['PL_Product_Category'] = df_final2['PL_Product_Category'].str.lower()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [207]:
df_final2['Product_Big_Series_Name'] = df_final2['Product_Big_Series_Name'].fillna('Others')
df_final2['Product_Line_Code_Name'] = df_final2['Product_Line_Code_Name'].fillna('Others')
df_final2['PL_Product_Category'] = df_final2['PL_Product_Category'].fillna('Others')
df_final2['Entitlement_Type'] = df_final2['Entitlement_Type'].fillna('Others')
df_final2['Entitlement_Status'] = df_final2['Entitlement_Type'].fillna('Others')
df_final2 = df_final2.dropna(subset=['Used & Consumed'])
df_final2.isna().sum()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Case_ID                       0
Product_Big_Series_Name       0
Product_Line_Code_Name        0
PL_Product_Category           0
Product_Number                0
Used & Consumed               0
Failure_Code                  0
Recommended_Parts             0
Serial_Number                 0
Entitlement_Type              0
Entitlement_Age_from_start    0
Entitlement_Age_from_end      0
Entitlement_Status            0
case_subject_refined          0
Closed_Date                   0
part_commodity                0
ip1                           0
ip2                           1
ip3                           1
dtype: int64

In [208]:
print(df_final2.shape)
df_final2 = df_final2[~(df_final2['Product_Big_Series_Name'].str.contains('printer|printers|supplies',case=False,regex=True))]
df_final2 = df_final2[~(df_final2['PL_Product_Category'].str.contains('printer|supplies|laser',case=False,regex=True))]
df_final2.shape

(1989, 19)


(1989, 19)

In [209]:
df_final2.PL_Product_Category.value_counts(normalize=True).cumsum()

commercial notebooks      0.946204
mobile workstations       0.979387
commercial accessories    0.990447
Others                    1.000000
Name: PL_Product_Category, dtype: float64

#### Dimensionality reduction on part number

In [210]:
pn_to_remove = df_final2.groupby(by=['Product_Number'])['Product_Line_Code_Name'].count().reset_index()
pn_remove = pn_to_remove[pn_to_remove['Product_Line_Code_Name']<10]['Product_Number'].unique().tolist()
final_pn_to_remove = ['Product_Number_'+x for x in pn_remove]
len(final_pn_to_remove)

601

#### Label encoding on Commodity

In [211]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = df_final2['part_commodity']
df_final2['commodity'] = df_final2['part_commodity']
df_final2['part_commodity'] = label_encoder.fit_transform(df_final2['part_commodity'])
print(df_final2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1989 entries, 0 to 3136
Data columns (total 20 columns):
Case_ID                       1989 non-null int64
Product_Big_Series_Name       1989 non-null object
Product_Line_Code_Name        1989 non-null object
PL_Product_Category           1989 non-null object
Product_Number                1989 non-null object
Used & Consumed               1989 non-null float64
Failure_Code                  1989 non-null object
Recommended_Parts             1989 non-null float64
Serial_Number                 1989 non-null object
Entitlement_Type              1989 non-null object
Entitlement_Age_from_start    1989 non-null int32
Entitlement_Age_from_end      1989 non-null int32
Entitlement_Status            1989 non-null object
case_subject_refined          1989 non-null object
Closed_Date                   1989 non-null int32
part_commodity                1989 non-null int64
ip1                           1989 non-null object
ip2                          

In [212]:
pickle.dump(label_encoder,open(f'F:\Part_Reco\Part_reco_regionwise\models\lbl_encoder_{current_region}.pkl','wb'))

### Inclusion of ip data

In [213]:
df_final2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1989 entries, 0 to 3136
Data columns (total 20 columns):
Case_ID                       1989 non-null int64
Product_Big_Series_Name       1989 non-null object
Product_Line_Code_Name        1989 non-null object
PL_Product_Category           1989 non-null object
Product_Number                1989 non-null object
Used & Consumed               1989 non-null float64
Failure_Code                  1989 non-null object
Recommended_Parts             1989 non-null float64
Serial_Number                 1989 non-null object
Entitlement_Type              1989 non-null object
Entitlement_Age_from_start    1989 non-null int32
Entitlement_Age_from_end      1989 non-null int32
Entitlement_Status            1989 non-null object
case_subject_refined          1989 non-null object
Closed_Date                   1989 non-null int32
part_commodity                1989 non-null int64
ip1                           1989 non-null object
ip2                          

In [214]:
print(df_final2.shape)
multilabel = pd.get_dummies(df_final2['commodity'])
df_final_ip = pd.concat([df_final2,multilabel], axis=1)
df_final_ip = df_final_ip.drop(columns=['part_commodity','commodity'])
print(df_final_ip.shape)

(1989, 20)
(1989, 33)


In [215]:
#list(multilabel.columns)
df_multilabel = df_final_ip.groupby(by=['Case_ID'])[multilabel.columns].sum()
for column in df_multilabel.columns:
    df_multilabel.loc[df_multilabel[column]>1,column] = 1
    
df_multilabel = df_multilabel.reset_index()

In [216]:
df_final_ip = df_final_ip.drop(columns=list(multilabel.columns))
df_final_ip.shape

(1989, 18)

In [217]:
df_final_ip = df_final_ip.merge(df_multilabel,how='left',on=['Case_ID'])
df_final_ip.shape

(1989, 33)

In [218]:
df_final_ip = df_final_ip.drop_duplicates()
df_final_ip.shape

(1743, 33)

In [219]:
#df_final_ip.to_csv('Multilabel_features.csv',index=False)

In [220]:

print(df_final_ip.shape, df_final_ip.isna().sum())

train_data = df_final_ip[~(df_final_ip['Closed_Date'].isin(test_period))]
test_data = df_final_ip[df_final_ip['Closed_Date'].isin(test_period)]
train_data = train_data[train_data['Used & Consumed']<=3]
train_data = train_data.drop(columns=['Used & Consumed','Failure_Code','Recommended_Parts','Serial_Number'])
test_data = test_data.drop(columns=['Used & Consumed','Failure_Code','Recommended_Parts','Serial_Number'])
test_cases_dt = test_data[['Case_ID']]

columns_onehot = list(train_data.columns)
columns_onehot.remove('case_subject_refined')

train_data_binarized = pd.get_dummies(train_data[columns_onehot])
train_data_binarized['case_subject_refined'] = train_data['case_subject_refined']

test_data_binarized = pd.get_dummies(test_data[columns_onehot])
test_data_binarized['case_subject_refined'] = test_data['case_subject_refined']

print(train_data_binarized.shape, test_data_binarized.shape)



(1743, 33) Case_ID                       0
Product_Big_Series_Name       0
Product_Line_Code_Name        0
PL_Product_Category           0
Product_Number                0
Used & Consumed               0
Failure_Code                  0
Recommended_Parts             0
Serial_Number                 0
Entitlement_Type              0
Entitlement_Age_from_start    0
Entitlement_Age_from_end      0
Entitlement_Status            0
case_subject_refined          0
Closed_Date                   0
ip1                           0
ip2                           1
ip3                           1
Battery                       0
Cable/Adaptr/PwrCord          0
Chassis/Enclosure             0
Display                       0
Electronic Component          0
Fan/Motor                     0
Hard Drive                    0
Human Input Device            0
Keyboard                      0
Memory/Firmware               0
Misc Hardware                 0
PCA I/O                       0
PCA Misc                     

In [221]:
df_final_ip['Closed_Date'].unique()

array([201911, 201912, 202001, 202003, 202002, 202004, 202005, 202006,
       202009, 202010, 202007, 202011, 202012, 202008, 202101],
      dtype=int64)

In [222]:
df_final_ip[(df_final_ip.ip1=='Not available')].head()

,Case_ID,Product_Big_Series_Name,Product_Line_Code_Name,PL_Product_Category,Product_Number,Used & Consumed,Failure_Code,Recommended_Parts,Serial_Number,Entitlement_Type,Entitlement_Age_from_start,Entitlement_Age_from_end,Entitlement_Status,case_subject_refined,Closed_Date,ip1,ip2,ip3,Battery,Cable/Adaptr/PwrCord,Chassis/Enclosure,Display,Electronic Component,Fan/Motor,Hard Drive,Human Input Device,Keyboard,Memory/Firmware,Misc Hardware,PCA I/O,PCA Misc,PwrSuply/Mains/Adptr,Solid State Drive
1966,5056581503,hp elitebook 840 g6 notebook pc (crescent14 1.0),AN,commercial notebooks,2J004UP,1.0,73 - Used & Consumed,1.0,5CG0166WZH,Care-Pack,6,-29,Care-Pack,no video,202011,Not available,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [223]:
 #df_final_ip[(df_final_ip.ip1=='Not available') & (df_final_ip['Closed_Date']>=201907)][multilabel.columns].sum()

#df_final_ip.ip1.value_counts()
df_final_ip[(df_final_ip.ip1=='Not available')].Closed_Date.value_counts().sort_index()

202011    1
Name: Closed_Date, dtype: int64

In [224]:
df_final_ip['ip1'] = df_final_ip.ip1.fillna('Not available')
df_final_ip['ip2'] = df_final_ip.ip1.fillna('Not available')
df_final_ip['ip3'] = df_final_ip.ip1.fillna('Not available')

In [225]:
remove_pns1 = list((train_data_binarized.columns[train_data_binarized.columns.str.contains('Product_Number')]).intersection(final_pn_to_remove))
train_data_binarized = train_data_binarized.drop(columns=remove_pns1)

remove_pns2 = list((test_data_binarized.columns[test_data_binarized.columns.str.contains('Product_Number')]).intersection(final_pn_to_remove))
test_data_binarized = test_data_binarized.drop(columns=remove_pns2)

new_features_in_test = set(test_data_binarized.columns) - set(train_data_binarized.columns)
len(new_features_in_test)

new_test = []
new_test_data = test_data_binarized.copy()
for feature in new_features_in_test:
    new_test.append(new_test_data[new_test_data[feature]==1])
    new_test_data = new_test_data[new_test_data[feature]!=1]
new_test_df = pd.concat(new_test)
print(new_test_df.shape,new_test_data.shape,test_data_binarized.shape)

test_data_binarized = pd.concat([new_test_data,pd.DataFrame(columns=train_data_binarized.columns,dtype='int64')],axis=0,ignore_index=True)
test_data_binarized = test_data_binarized[train_data_binarized.columns]

new_test_binarized = pd.concat([new_test_df,pd.DataFrame(columns=train_data_binarized.columns,dtype='int64')],axis=0,ignore_index=True)
new_test_binarized = new_test_binarized[train_data_binarized.columns]

print(train_data_binarized.shape, test_data_binarized.shape, new_test_binarized.shape)



(14, 211) (413, 211) (427, 211)
(1307, 273) (413, 273) (14, 273)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [226]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=1500)
train_vect = vectorizer.fit_transform(train_data_binarized['case_subject_refined'])
test_vect = vectorizer.transform(test_data_binarized['case_subject_refined'])
new_test_vect = vectorizer.transform(new_test_binarized['case_subject_refined'])

train_matrix = pd.DataFrame(data = train_vect.toarray(), columns = vectorizer.get_feature_names())
test_matrix = pd.DataFrame(data = test_vect.toarray(), columns = vectorizer.get_feature_names())
new_test_matrix = pd.DataFrame(data = new_test_vect.toarray(), columns = vectorizer.get_feature_names())

final_train_data = pd.concat([train_data_binarized.reset_index(drop=True),train_matrix],axis=1)
final_test_data = pd.concat([test_data_binarized.reset_index(drop=True),test_matrix],axis=1)
final_new_test_data = pd.concat([new_test_binarized.reset_index(drop=True),new_test_matrix],axis=1)
final_test_data = final_test_data.fillna(0)
final_new_test_data = final_new_test_data.fillna(0)

final_train_data = final_train_data.drop(columns=['Closed_Date','case_subject_refined'])
final_test_data = final_test_data.drop(columns=['Closed_Date','case_subject_refined'])
final_new_test_data = final_new_test_data.drop(columns=['Closed_Date','case_subject_refined'])

print(df_final2.shape, final_train_data.shape, final_test_data.shape, final_new_test_data.shape)


(1989, 20) (1307, 642) (413, 642) (14, 642)


In [227]:
pickle.dump(vectorizer,  open(f'F:\\Part_Reco\\Part_reco_regionwise\\models\\vectorizer_{current_region}.pkl', "wb"))

In [228]:

combined_data = pd.concat([final_train_data, final_test_data])

combined_data = combined_data.reset_index(drop=True)

y_data = combined_data[multilabel.columns]
X_data = combined_data.drop(columns=multilabel.columns)

train_X, train_y, test_X, test_y = iterative_train_test_split(X_data.values,y_data.values,test_size=0.2)
train_X = pd.DataFrame(train_X, columns=X_data.columns)
test_X = pd.DataFrame(test_X, columns=X_data.columns)
train_y = pd.DataFrame(train_y, columns=y_data.columns)
test_y = pd.DataFrame(test_y, columns=y_data.columns)

train_X_ann, train_y_ann, val_X_ann, val_y_ann = iterative_train_test_split(train_X.values,train_y.values,test_size=0.2)
train_X_ann = pd.DataFrame(train_X_ann, columns=train_X.columns)
val_X_ann = pd.DataFrame(val_X_ann, columns=train_X.columns)
train_y_ann = pd.DataFrame(train_y_ann, columns=train_y.columns)
val_y_ann = pd.DataFrame(val_y_ann, columns=train_y.columns)

test_cases = test_X[['Case_ID']]
train_cases = train_X[['Case_ID']]

train_X = train_X.drop(columns=['Case_ID'])
test_X = test_X.drop(columns=['Case_ID'])
train_X_ann = train_X_ann.drop(columns=['Case_ID'])
val_X_ann = val_X_ann.drop(columns=['Case_ID'])



In [229]:
train_X.shape

(1378, 626)

In [230]:
pickle.dump(train_X.columns, open(f"F:\\Part_Reco\\Part_reco_regionwise\\models\\train_data_{current_region}.pkl", "wb"))

In [231]:
del combined_data
gc.collect()

60

In [232]:
del X_data, y_data
gc.collect()

20

In [233]:
train_X_sparse = csc_matrix(train_X)
test_X_sparse = csc_matrix(test_X)

#### Classification using train_test_split

In [234]:
'''
logclassifier1 = LogisticRegression(n_jobs=-1, solver='lbfgs', max_iter=300)
logclassifier1.fit(train_X_sparse,train_y)
pred_test1= (logclassifier1.predict(test_X_sparse)).astype('int')


#print('Confusion matrix', confusion_matrix(y_test,pred3))
print('Precision', precision_score(test_y,pred_test1,average='weighted'))
print('Recall', recall_score(test_y,pred_test1,average='weighted'))
print('Accuracy', accuracy_score(test_y,pred_test1)) #Accuracy 0.7209060378185004
print('Classification report', classification_report(test_y,pred_test1))
'''

"\nlogclassifier1 = LogisticRegression(n_jobs=-1, solver='lbfgs', max_iter=300)\nlogclassifier1.fit(train_X_sparse,train_y)\npred_test1= (logclassifier1.predict(test_X_sparse)).astype('int')\n\n\n#print('Confusion matrix', confusion_matrix(y_test,pred3))\nprint('Precision', precision_score(test_y,pred_test1,average='weighted'))\nprint('Recall', recall_score(test_y,pred_test1,average='weighted'))\nprint('Accuracy', accuracy_score(test_y,pred_test1)) #Accuracy 0.7209060378185004\nprint('Classification report', classification_report(test_y,pred_test1))\n"

#### Classification using Date period split

In [235]:
y_train_dt = final_train_data[multilabel.columns] 
X_train_dt = final_train_data.drop(columns=multilabel.columns)

y_test_dt = final_test_data[multilabel.columns] 
X_test_dt = final_test_data.drop(columns=multilabel.columns)

X_train_dt = X_train_dt.drop(columns=['Case_ID'])
X_test_dt = X_test_dt.drop(columns=['Case_ID'])

In [236]:

X_train_dt_sparse = csc_matrix(X_train_dt)
X_test_dt_sparse = csc_matrix(X_test_dt)

logclassifier2 = OneVsRestClassifier(LogisticRegression(n_jobs=-1, solver='lbfgs', max_iter=300))
logclassifier2.fit(X_train_dt_sparse,y_train_dt)
pred_test2= (logclassifier2.predict(X_test_dt_sparse)).astype('int')


#print('Confusion matrix', confusion_matrix(y_test,pred3))
#print('Precision', precision_score(y_test_dt,pred_test2,average='weighted'))
#print('Recall', recall_score(y_test_dt,pred_test2,average='weighted'))
print('Accuracy', accuracy_score(y_test_dt,pred_test2)) #Accuracy 0.7063969382176053
print('Classification report', classification_report(y_test_dt,pred_test2))

Accuracy 0.8038740920096852
Classification report              precision    recall  f1-score   support

          0       0.97      0.96      0.97        76
          1       0.00      0.00      0.00        14
          2       0.00      0.00      0.00         8
          3       0.92      0.58      0.71        19
          4       0.75      0.38      0.50         8
          5       1.00      1.00      1.00        17
          6       0.93      0.93      0.93        54
          7       0.88      0.58      0.70        12
          8       0.99      0.93      0.96       107
          9       0.80      0.50      0.62         8
         10       0.00      0.00      0.00         2
         11       0.97      0.79      0.87        94
         12       0.00      0.00      0.00         4
         13       1.00      0.77      0.87        26
         14       1.00      0.64      0.78        14

avg / total       0.90      0.79      0.84       463



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [237]:
pickle.dump(logclassifier2,open(f'F:\\Part_Reco\\Part_reco_regionwise\\models\\logistic_model_{current_region}.pkl',"wb"))

In [238]:
#Deep learning initialization

units = 300
units2 = 70
kernel_initializer = 'lecun_normal'
activation ='elu'
drop_rate1 = 0.1
drop_rate2 = 0.3
drop_rate3 = 0.1
activation_output = 'sigmoid'
optimizer = 'rmsprop'
loss = 'binary_crossentropy'
metrics = ['categorical_accuracy']
n_epochs = 20
batch_size = 32

In [239]:
#Deep learning model

np.random.seed(5)
input_tensor = Input(shape=(train_X_ann.shape[1],))

x = layers.Dense(units=units,input_dim=list(train_X_ann.shape)[1],kernel_initializer=kernel_initializer)(input_tensor)
y = layers.BatchNormalization()(x)
y = layers.Activation(activation)(y)
y = layers.Dropout(rate=drop_rate1)(y)

y = layers.Dense(units=units/2, kernel_initializer=kernel_initializer)(y)
y = layers.BatchNormalization()(y)
y = layers.Activation(activation)(y)
y = layers.Dropout(rate=drop_rate2)(y)

y = layers.Dense(units=units, kernel_initializer=kernel_initializer)(y)
y = layers.BatchNormalization()(y)
y = layers.Activation(activation)(y)
y = layers.Dropout(rate=drop_rate1)(y)

y = layers.add([y, x])

y = layers.Dense(units=units2, kernel_initializer=kernel_initializer)(y)
y = layers.BatchNormalization()(y)
y = layers.Activation(activation)(y)
y = layers.Dropout(rate=drop_rate3)(y)

y = layers.Dense(units=np.int(units2*0.7), kernel_initializer=kernel_initializer)(y)
y = layers.BatchNormalization()(y)
y = layers.Activation(activation)(y)
y = layers.Dropout(rate=drop_rate3)(y)

output_tensor = layers.Dense(units=train_y.shape[1], kernel_initializer=kernel_initializer, activation=activation_output)(y)

classifier1 = Model(input_tensor, output_tensor)
classifier2 = Model(input_tensor, output_tensor)

In [240]:
X_train_dt_ann, y_train_dt_ann, X_val_dt_ann , y_val_dt_ann = iterative_train_test_split(X_train_dt.values,y_train_dt.values,test_size=0.2)

In [241]:

classifier2.compile(optimizer=optimizer, loss=loss, metrics=metrics)
classifier2.fit(x=X_train_dt_ann, y=y_train_dt_ann, batch_size=batch_size, epochs=n_epochs, validation_data=(X_val_dt_ann, y_val_dt_ann),
               callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=2,min_lr=0)])

pred_ann_test2 = classifier2.predict(X_test_dt)

Train on 1045 samples, validate on 262 samples
Epoch 1/20
1045/1045 [==============================] - 1s 1ms/sample - loss: 0.6285 - categorical_accuracy: 0.4172 - val_loss: 0.5517 - val_categorical_accuracy: 0.4160
Epoch 2/20
1045/1045 [==============================] - 0s 211us/sample - loss: 0.4750 - categorical_accuracy: 0.6287 - val_loss: 0.4042 - val_categorical_accuracy: 0.3397
Epoch 3/20
1045/1045 [==============================] - 0s 210us/sample - loss: 0.3554 - categorical_accuracy: 0.7005 - val_loss: 0.2944 - val_categorical_accuracy: 0.6221
Epoch 4/20
1045/1045 [==============================] - 0s 211us/sample - loss: 0.2639 - categorical_accuracy: 0.7330 - val_loss: 0.2630 - val_categorical_accuracy: 0.6489
Epoch 5/20
1045/1045 [==============================] - 0s 211us/sample - loss: 0.1928 - categorical_accuracy: 0.7550 - val_loss: 0.1883 - val_categorical_accuracy: 0.6985
Epoch 6/20
1045/1045 [==============================] - 0s 212us/sample - loss: 0.1482 - catego

In [242]:
print('Test set Accuracy', accuracy_score(y_test_dt.values,pred_ann_test2.round())) #Test set Accuracy 0.7427556041552761
print('Classification report', classification_report(y_test_dt.values,pred_ann_test2.round()))

Test set Accuracy 0.7941888619854721
Classification report              precision    recall  f1-score   support

          0       0.96      0.84      0.90        76
          1       0.00      0.00      0.00        14
          2       0.17      0.25      0.20         8
          3       0.93      0.68      0.79        19
          4       0.80      0.50      0.62         8
          5       1.00      1.00      1.00        17
          6       0.93      0.96      0.95        54
          7       0.83      0.42      0.56        12
          8       0.99      0.93      0.96       107
          9       0.83      0.62      0.71         8
         10       0.00      0.00      0.00         2
         11       0.92      0.78      0.84        94
         12       0.00      0.00      0.00         4
         13       1.00      0.92      0.96        26
         14       0.91      0.71      0.80        14

avg / total       0.89      0.79      0.84       463



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [243]:
pickle.dump(y_test_dt.columns, open(f'F:\\Part_Reco\\Part_reco_regionwise\\models\\labels_{current_region}.pkl', "wb"))

In [244]:
ann_model = classifier2.to_json()
with open(f'F:\\Part_Reco\\Part_reco_regionwise\\models\\ann_model_{current_region}.json', "w") as json_file:
    json_file.write(ann_model)
# serialize weights to HDF5
classifier2.save_weights(f'F:\\Part_Reco\\Part_reco_regionwise\\models\\ann_model_{current_region}.h5')

In [245]:
gc.collect()

39

In [246]:
xgb2 = OneVsRestClassifier(XGBClassifier(max_depth=3,n_jobs=-1,n_estimators=500,subsample=0.6, colsample_bytree=0.8))
xgb2.fit(X_train_dt_sparse,y_train_dt)
pred_test4= (xgb2.predict(X_test_dt_sparse)).astype('int')


#print('Confusion matrix', confusion_matrix(y_test,pred3))
#print('Precision', precision_score(y_test_dt,pred_test4,average='weighted'))
#print('Recall', recall_score(y_test_dt,pred_test4,average='weighted'))
print('Accuracy', accuracy_score(y_test_dt,pred_test4))
print('Classification report', classification_report(y_test_dt,pred_test4))
#Accuracy 0.7236194641880809

Accuracy 0.7602905569007264
Classification report              precision    recall  f1-score   support

          0       0.99      0.93      0.96        76
          1       0.33      0.14      0.20        14
          2       1.00      0.12      0.22         8
          3       0.92      0.58      0.71        19
          4       1.00      0.38      0.55         8
          5       1.00      0.94      0.97        17
          6       0.87      0.89      0.88        54
          7       0.86      0.50      0.63        12
          8       0.98      0.93      0.95       107
          9       0.60      0.38      0.46         8
         10       0.00      0.00      0.00         2
         11       0.89      0.74      0.81        94
         12       0.00      0.00      0.00         4
         13       0.95      0.77      0.85        26
         14       0.90      0.64      0.75        14

avg / total       0.90      0.78      0.82       463



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [247]:
pickle.dump(xgb2,open(f'F:\\Part_Reco\\Part_reco_regionwise\\models\\xgb_model_{current_region}.pkl',"wb"))

#### Ensemble method

In [248]:
logis_en2 = logclassifier2.predict_proba(X_test_dt_sparse)
xgb_en2 = xgb2.predict_proba(X_test_dt_sparse)
ann_en2 = classifier2.predict(X_test_dt)


In [249]:
#ensem_output = (logis_arr*0.5+xgb_arr*0.5+ann_en2*2)/3
#ensem_output = np.maximum(logis_arr,xgb_arr,ann_en2)
ensem_output = np.maximum(logis_en2,xgb_en2,ann_en2)


print('Accuracy', accuracy_score(y_test_dt,ensem_output.round()))
print('Classification report', classification_report(y_test_dt,ensem_output.round()))
#Accuracy 0.7843868945443689

Accuracy 0.7966101694915254
Classification report              precision    recall  f1-score   support

          0       0.97      0.97      0.97        76
          1       0.33      0.14      0.20        14
          2       0.50      0.12      0.20         8
          3       0.93      0.68      0.79        19
          4       0.75      0.38      0.50         8
          5       1.00      1.00      1.00        17
          6       0.86      0.93      0.89        54
          7       0.89      0.67      0.76        12
          8       0.98      0.95      0.97       107
          9       0.67      0.50      0.57         8
         10       0.00      0.00      0.00         2
         11       0.89      0.79      0.84        94
         12       0.00      0.00      0.00         4
         13       0.95      0.81      0.88        26
         14       0.92      0.86      0.89        14

avg / total       0.89      0.82      0.85       463



In [250]:
gc.collect()

171

In [251]:
test_cases_dt = df_final_ip[df_final_ip['Closed_Date'].isin(test_period)]
test_cases_dt = test_cases_dt[~test_cases_dt.Case_ID.isin(list(new_test_binarized.Case_ID.values))]

In [252]:
max_prob = np.around(np.take_along_axis(np.array(pred_ann_test2),np.argsort(-np.array(pred_ann_test2),axis=1),axis=-1)[:,:5]*100,2)
pred_comm = y_test_dt.columns[np.argsort(-np.array(pred_ann_test2),axis=1)[:,:5]]

In [253]:
#test_cases_dt['part_commodity'] = label_encoder.inverse_transform(test_cases_dt['part_commodity'])
prediction_df = pd.concat([pd.DataFrame(max_prob,columns=['confidence_'+str(i) for i in np.arange(1,6)]),pd.DataFrame(pred_comm,columns=['commodity_'+str(i) for i in np.arange(1,6)])],axis=1)
Final_prediction = pd.concat([test_cases_dt.reset_index(drop=True),prediction_df],axis=1)
part_mapping = pd.read_csv(f'F:\\Akhilesh_new\\Part_reco_regionwise\\Part_number_mapping_{date}_{current_region}.csv')
part_prediction = Final_prediction.merge(part_mapping[['Product_Number','part_commodity','Overall_High_Demand_Part','Product_level_High_Demand_Part']]
                                         .rename(columns={'part_commodity':'commodity_1','Product_level_High_Demand_Part':'Part_1','Overall_High_Demand_Part':'Sub_Part_1'}),
                                        how='left',on=['Product_Number','commodity_1'])

part_prediction = part_prediction.merge(part_mapping[['Product_Number','part_commodity','Overall_High_Demand_Part','Product_level_High_Demand_Part']]
                                         .rename(columns={'part_commodity':'commodity_2','Product_level_High_Demand_Part':'Part_2','Overall_High_Demand_Part':'Sub_Part_2'}),
                                        how='left',on=['Product_Number','commodity_2'])

part_prediction = part_prediction.merge(part_mapping[['Product_Number','part_commodity','Overall_High_Demand_Part','Product_level_High_Demand_Part']]
                                         .rename(columns={'part_commodity':'commodity_3','Product_level_High_Demand_Part':'Part_3','Overall_High_Demand_Part':'Sub_Part_3'}),
                                        how='left',on=['Product_Number','commodity_3'])

part_prediction = part_prediction.merge(part_mapping[['Product_Number','part_commodity','Overall_High_Demand_Part','Product_level_High_Demand_Part']]
                                         .rename(columns={'part_commodity':'commodity_4','Product_level_High_Demand_Part':'Part_4','Overall_High_Demand_Part':'Sub_Part_4'}),
                                        how='left',on=['Product_Number','commodity_4'])

part_prediction = part_prediction.merge(part_mapping[['Product_Number','part_commodity','Overall_High_Demand_Part','Product_level_High_Demand_Part']]
                                         .rename(columns={'part_commodity':'commodity_5','Product_level_High_Demand_Part':'Part_5','Overall_High_Demand_Part':'Sub_Part_5'}),
                                        how='left',on=['Product_Number','commodity_5'])

part_prediction.to_csv(f'F:\\Part_Reco\\Part_reco_regionwise\\models\\Prediction_data_{date}_{current_region}.csv',index=False)

In [254]:

t2=t.clock()
print('Total time taken in hours: ', round((t2-t1)/60/60,2))

Total time taken in hours:  0.03


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.
